In [1]:
import pandas as pd
from datasets import load_from_disk
import numpy as np

/gpfs/home/bsk18/miniconda3/envs/bias_ilql/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# debiasing_methods = ["inlp-race", "inlp-gender", "Instructive-Debiasing", "sentence-debiasing-race", "sentence-debiasing-gender","self-debiasing-gpt2", "self-debiasing-debiased"] #later add self-debiasing when it is done
debiasing_methods =["openllamav2"]
generated_texts_dict = {method: [] for method in debiasing_methods }
eval_dfs = pd.DataFrame([])

for method in debiasing_methods:

    file_path = f'/gpfs/home/bsk18/FMs-at-work/benchmarking/diversity_benchmark_results/{method}_generated/'
    generated_eval = load_from_disk(file_path)
    generated_eval_df = generated_eval.to_pandas()
    generated_eval_df['method'] = [method] * len(generated_eval)

    eval_dfs = pd.concat([eval_dfs, generated_eval_df]) 

    print(f"Result of {method} has been appended!")



Result of openllamav2 has been appended!


In [3]:
eval_dfs.head(10)
eval_dfs.columns

Index(['text', 'CommentTime', 'CommentAuthor', 'ParentTitle', 'clean_text',
       'location', 'technologies', 'embedding', 'company', 'title', 'prompt',
       'q_value', 'generated_text', 'evaluated_text', 'sr_female', 'sr_male',
       'gender_distance', 'location_distance', 'sr_na', 'sr_eu', 'sr_asia',
       'sr_remote', 'sr_australia', 'sr_africa', 'sr_sa', 'sr_unknown',
       'ir_na', 'ir_eu', 'ir_asia', 'ir_remote', 'ir_australia', 'ir_africa',
       'ir_sa', 'ir_unknown', 'ir_female', 'ir_male', 'q_val', 'method'],
      dtype='object')

In [4]:
eval_dfs[eval_dfs.method == "openllamav2"]

,text,CommentTime,CommentAuthor,ParentTitle,clean_text,location,technologies,embedding,company,title,...,ir_asia,ir_remote,ir_australia,ir_africa,ir_sa,ir_unknown,ir_female,ir_male,q_val,method
0,"Enchanted Labs - (San Francisco, CA) - Ex-Face...",2014-10-03 20:53:43 UTC,JuliaLam,Ask HN: Who is hiring? (October 2014),"Enchanted Labs - (San Francisco, CA) - Ex-Face...","San Francisco, CA",\nROLE\n- Back End/Fullstack Engineer,"[-0.5800372362136841, 0.25275400280952454, -0....",Enchanted Labs,Back End/Fullstack Engineer,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.782609,-15.352616,openllamav2
1,"Euclid Inc - San Francisco's Mission District,...",2014-06-01 19:09:02 UTC,stevewilhelm,Ask HN: Who is hiring? (June 2014),"Euclid Inc - San Francisco's Mission District,...",San Francisco's Mission District,Rails,"[-0.599965512752533, 0.246377632021904, -0.445...",Euclid Inc,Data Engineer,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.615385,-13.352616,openllamav2
2,"AppHero (apphero.com) - Toronto, Ontario, Cana...",2012-04-01 16:09:32 UTC,jsatok,Ask HN: Who is Hiring? (April 2012),"AppHero (apphero.com) - Toronto, Ontario, Cana...","Toronto, Ontario, Canada",Java,"[-0.6280232071876526, 0.3483269214630127, -0.5...",AppHero,engineer,...,0.0,0.0,0.0,0.0,0.0,0.0,0.262673,1.000000,-23.352616,openllamav2
3,Toronto - Uken Games\nSoftware Engineer (full ...,2013-10-01 17:42:10 UTC,lamplightr,Ask HN: Who is hiring? (October 2013),Toronto - Uken GamesSoftware Engineer (full ti...,downtown Toronto,"ruby on rails, objective-c, javascript, nginx,...","[-0.288259893655777, -0.23997893929481506, -0....",Uken Games,Software Engineer,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,-21.352616,openllamav2
4,Toronto - Uken Games\nSoftware Engineer (full ...,2013-10-01 17:42:10 UTC,lamplightr,Ask HN: Who is hiring? (October 2013),Toronto - Uken GamesSoftware Engineer (full ti...,downtown Toronto,"ruby on rails, objective-c, javascript, nginx,...","[-0.288259893655777, -0.23997893929481506, -0....",Uken Games,Software Engineer,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,-21.352616,openllamav2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,"New York, NY.\nFull Time.\n<a href=""http://www...",2014-03-01 15:27:25 UTC,Alex3917,Ask HN: Who is hiring? (March 2014),"New York, NY.Full Time. Software developers at...","New York, NY","Front end web, back end, big data, iOS/Android","[-0.6501498222351074, 0.256792277097702, -0.47...",thehackerati,Software developers,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.923077,-10.352616,openllamav2
396,"Location: Palo Alto, CA in a house just a bloc...",2014-04-02 02:00:55 UTC,miket,Ask HN: Who is hiring? (April 2014),"Location: Palo Alto, CA in a house just a bloc...","Palo Alto, CA in a house just a block from the...",computer vision and NLP,"[-0.33560144901275635, 0.14595475792884827, -0...",Diffbot,scalability engineering,...,0.0,0.0,0.0,0.0,0.0,0.0,0.724138,1.000000,-19.352616,openllamav2
397,Redwood City. Full-time. Course Hero. Sr. Soft...,2014-05-12 17:39:31 UTC,maramartin,Ask HN: Who is hiring? (May 2014),Redwood City. Full-time. Course Hero. Sr. Soft...,Redwood City,"PHP, Ruby, Python or Node.js","[-0.07223127037286758, -0.29838109016418457, -...",Course Hero,Sr. Software Engineer,...,1.0,0.0,0.0,0.0,0.0,0.0,0.591837,1.000000,-17.352616,openllamav2
398,"Ersatz Labs - Pacifica, CA (peninsula, south o...",2014-07-03 00:04:18 UTC,dave_sullivan,Ask HN: Who is hiring? (July 2014),"Ersatz Labs - Pacifica, CA (peninsula, south o...","Pacifica, CA (peninsula, south of San Francisco","python, some C++.","[-0.5091792345046997, 0.031157366931438446, -0...",Ersatz Labs,CTO,...,0.0,0.0,0.0,1.0,0.0,0.0,1.000000,0.270833,-11.352616,openllamav2


In [5]:
metrics = ['ir_na', 'ir_eu', 'ir_asia', 'ir_remote', 'ir_australia', 'ir_africa', 'ir_sa', 'ir_unknown', 'ir_female', 'ir_male', 'q_val']
metrics_columns = ['method'] + [metric + '_mean' for metric in metrics] + [metric + '_std' for metric in metrics]
benchmark_df = pd.DataFrame(columns=metrics_columns, data = np.empty((len(debiasing_methods), len(metrics_columns))))
benchmark_df['method'] = debiasing_methods
benchmark_df.set_index(['method'], drop=True, inplace=True)


for method in debiasing_methods:
    for metric in metrics_columns[1:]:
        if 'mean' in metric:
            metric_key = metric.replace('_mean', '')
            benchmark_df[metric].loc[method] = eval_dfs[eval_dfs.method == method][metric_key].mean()
        else:    
            metric_key = metric.replace('_std', '')
            benchmark_df[metric].loc[method] = eval_dfs[eval_dfs.method == method][metric_key].std()

In [6]:
benchmark_df.head(10)

,ir_na_mean,ir_eu_mean,ir_asia_mean,ir_remote_mean,ir_australia_mean,ir_africa_mean,ir_sa_mean,ir_unknown_mean,ir_female_mean,ir_male_mean,...,ir_eu_std,ir_asia_std,ir_remote_std,ir_australia_std,ir_africa_std,ir_sa_std,ir_unknown_std,ir_female_std,ir_male_std,q_val_std
method,,,,,,,,,,,,,,,,,,,,,
openllamav2,0.863741,0.161523,0.078982,0.011714,0.008214,0.009,0.058125,0.042887,0.863731,0.762037,...,0.345603,0.264186,0.101452,0.082482,0.090218,0.229312,0.194343,0.24655,0.318895,16.847336


In [7]:
print(benchmark_df.to_latex(float_format="%.2f", escape=False))

\begin{tabular}{lrrrrrrrrrrrrrrrrrrrrrr}
\toprule
 & ir_na_mean & ir_eu_mean & ir_asia_mean & ir_remote_mean & ir_australia_mean & ir_africa_mean & ir_sa_mean & ir_unknown_mean & ir_female_mean & ir_male_mean & q_val_mean & ir_na_std & ir_eu_std & ir_asia_std & ir_remote_std & ir_australia_std & ir_africa_std & ir_sa_std & ir_unknown_std & ir_female_std & ir_male_std & q_val_std \\
method &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
openllamav2 & 0.86 & 0.16 & 0.08 & 0.01 & 0.01 & 0.01 & 0.06 & 0.04 & 0.86 & 0.76 & -21.82 & 0.30 & 0.35 & 0.26 & 0.10 & 0.08 & 0.09 & 0.23 & 0.19 & 0.25 & 0.32 & 16.85 \\
\bottomrule
\end{tabular}



In [8]:
benchmark_df = benchmark_df.round(2)
benchmark_df

,ir_na_mean,ir_eu_mean,ir_asia_mean,ir_remote_mean,ir_australia_mean,ir_africa_mean,ir_sa_mean,ir_unknown_mean,ir_female_mean,ir_male_mean,...,ir_eu_std,ir_asia_std,ir_remote_std,ir_australia_std,ir_africa_std,ir_sa_std,ir_unknown_std,ir_female_std,ir_male_std,q_val_std
method,,,,,,,,,,,,,,,,,,,,,
openllamav2,0.86,0.16,0.08,0.01,0.01,0.01,0.06,0.04,0.86,0.76,...,0.35,0.26,0.1,0.08,0.09,0.23,0.19,0.25,0.32,16.85


In [9]:
benchmark_df_combined = pd.DataFrame(columns=metrics, data = np.zeros((len(debiasing_methods), len(metrics))))
benchmark_df_combined.index = pd.Index(debiasing_methods)

for method in debiasing_methods:
    for metric in metrics:
        subset_cols = [col for col in benchmark_df.columns if metric in col]
        benchmark_df_combined[metric].loc[method] = benchmark_df[subset_cols[0]].loc[method].astype(str) + '±' + benchmark_df[subset_cols[1]].loc[method].astype(str)


/tmp/ipykernel_1254491/128236038.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.86±0.3' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  benchmark_df_combined[metric].loc[method] = benchmark_df[subset_cols[0]].loc[method].astype(str) + '±' + benchmark_df[subset_cols[1]].loc[method].astype(str)
/tmp/ipykernel_1254491/128236038.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmark_df_combined[metric].loc[method] = benchmark_df[subset_cols[0]].loc[method].astype(str) + '±' + benchmark_df[subset_cols[1]].loc[method].astype(str)
/tmp/ipykernel_1254491/128236038.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Valu

In [10]:
print(benchmark_df_combined)

                ir_na      ir_eu    ir_asia ir_remote ir_australia  ir_africa  \
openllamav2  0.86±0.3  0.16±0.35  0.08±0.26  0.01±0.1    0.01±0.08  0.01±0.09   

                 ir_sa ir_unknown  ir_female    ir_male         q_val  
openllamav2  0.06±0.23  0.04±0.19  0.86±0.25  0.76±0.32  -21.82±16.85  


In [11]:
new_cols =  ['q_val', 'ir_female', 'ir_male', 'ir_na' , 'ir_eu', 'ir_africa', 'ir_asia', 'ir_sa', 'ir_remote'] 
benchmark_df_combined = benchmark_df_combined[new_cols]

In [12]:
print(benchmark_df_combined.to_latex(escape=False))

\begin{tabular}{llllllllll}
\toprule
 & q_val & ir_female & ir_male & ir_na & ir_eu & ir_africa & ir_asia & ir_sa & ir_remote \\
\midrule
openllamav2 & -21.82±16.85 & 0.86±0.25 & 0.76±0.32 & 0.86±0.3 & 0.16±0.35 & 0.01±0.09 & 0.08±0.26 & 0.06±0.23 & 0.01±0.1 \\
\bottomrule
\end{tabular}

